# Neural Network based on the lichess puzzle database
Since the full puzzle database is over 700MB, I've elected to store the first 500,000 on Google Drive and then provide a download link where you can then store it in this project directory (*.csv is in gitignore).

The Lichess puzzle database consists of almost 3.9 million chess puzzles fetched from the lichess.org puzzle database. Each row contains a separate puzzle, which consists of its id, its initial setup (FEN), the move sequence, the best move, and the puzzle rating, along with a bit more information for use.

lichess_db_puzzle_500.csv -> https://drive.google.com/file/d/1OLWvis002H9oujoCNl4q9y4eSQxKq3hk/view?usp=sharing

I will say that I've made a small change to the original database in that the correct move sequence actually begins with the opponent's move first (the mistake/blunder in the position). I've reconfigured the FENs to have the first move already played and the best move on its own column.

Let's read in the CSV into a pandas dataframe and check it out:

In [2]:
import pandas as pd

dataset = pd.read_csv("lichess_db_puzzle_500.csv")

In [3]:
dataset.head()

,PuzzleId,FEN,Moves,Rating,RatingDeviation,Popularity,NbPlays,Themes,GameUrl,OpeningTags
0,00008,r6k/pp2r2p/4Rp1Q/3p4/8/1N1P2R1/PqP2bPP/7K b - ...,f2g3 e6e7 b2b1 b3c1 b1c1 h6c1,1917,76,94,6044,crushing hangingPiece long middlegame,https://lichess.org/787zsVup/black#47,NaN
1,0000D,5rk1/1p3ppp/pq3b2/8/8/1P1Q1N2/P4PPP/3R2K1 w - ...,d3d6 f8d8 d6d8 f6d8,1485,74,96,25966,advantage endgame short,https://lichess.org/F8M8OS71#52,NaN
2,0008Q,8/4R3/1p2P3/p4r2/P6p/1P3Pk1/4K3/8 w - - 1 64,e7f7 f5e5 e2f1 e5e6,1420,74,90,541,advantage endgame rookEndgame short,https://lichess.org/MQSyb3KW#126,NaN
3,0009B,r2qr1k1/b1p2ppp/pp4n1/P1P1p3/4P1n1/B2P2Pb/3NBP...,b6c5 e2g4 h3g4 d1g4,1123,74,87,567,advantage middlegame short,https://lichess.org/4MWQCxQ6/black#31,Kings_Pawn_Game Kings_Pawn_Game_Leonardis_Vari...
4,000Vc,8/8/4k1p1/2KpP2p/5PP1/8/8/8 w - - 0 53,g4h5 g6h5 f4f5 e6e5 f5f6 e5f6,1556,81,73,91,crushing endgame long pawnEndgame,https://lichess.org/l6AejDMO#104,NaN
